In [25]:
import pde
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# SIR model with diffusive mobility 
based on https://py-pde.readthedocs.io/en/latest/examples_gallery/pde_sir.html

In [26]:
SUSCEPTIBLE = "(S) Podatny"
INFECTIOUS = "(I) Zainfekowany"
RECOVERED = "(R) Wyleczony"

class SIR(pde.PDEBase):
    """SIR model with diffusive mobility"""

    def __init__(self, beta=0.4, gamma=0.04, diffusion=1.0, bc="auto_periodic_neumann"):
        """
            beta: transmission rate
            gamma: recovery rate
            diffusion: spatial mobility
            bc: boundary condition
        """
        super().__init__()
        self.beta = beta
        self.gamma = gamma
        self.diffusion = diffusion
        self.bc = bc

    def get_initial_state(self, s: pde.ScalarField, i: pde.ScalarField)->pde.FieldCollection:
        """get initial state"""
        norm = (s + i).data.max()  # maximal density
        if norm > 1:
            s /= norm
            i /= norm

        # create recovered field
        r = pde.ScalarField(s.grid, data=1 - s - i, label=RECOVERED)
        return pde.FieldCollection([s, i, r])

    def evolution_rate(self, state, t=0):
        s, i, r = state
        ds_dt = self.diffusion * s.laplace(self.bc) - self.beta * i * s
        di_dt = self.diffusion * i.laplace(self.bc) + self.beta * i * s - self.gamma * i
        dr_dt = self.diffusion * r.laplace(self.bc) + self.gamma * i
        return pde.FieldCollection([ds_dt, di_dt, dr_dt])

# Simulation plotting parameters

In [27]:
plot_1D_args={'ax_style': {'xlim': (0, 150), 'ylim': (0, 1)}}
plot_2D_args={"vmin": 0, "vmax": 1,'cmap':'coolwarm'}

# SIR 1D

In [28]:
equation = SIR(beta=0.4, gamma=0.04, diffusion=1)
# initialize state
grid = pde.UnitGrid([150])
s = pde.ScalarField(grid=grid, data=1, label=SUSCEPTIBLE)
i = pde.ScalarField(grid=grid, data=0, label=INFECTIOUS)
s.data[0] = 0
i.data[0] = 1
state = equation.get_initial_state(s, i)

storage = pde.MemoryStorage()
tracker = pde.PlotTracker(interval=10, title= "SIR 1D",plot_args=plot_1D_args)
solution = equation.solve(state, t_range=100, dt=1e-1, tracker=["progress", tracker, storage.tracker(interval=1)])

  0%|          | 0/100.0 [00:00<?, ?it/s]

Output()

Spent more time on handling trackers (2.36) than on the actual simulation (0.344)


In [29]:
#save simulation as movie
pde.movie(storage=storage, filename='./movies/pde_sir_1D.mp4', progress=True, show_time=True, plot_args=plot_1D_args)

  0%|          | 0/101 [00:00<?, ?it/s]

# SIR 2D

In [30]:
equation = SIR(beta=0.4, gamma=0.04, diffusion=1)
# initialize state
grid = pde.UnitGrid([100, 100])
s = pde.ScalarField(grid=grid, data=1, label=SUSCEPTIBLE)
i = pde.ScalarField(grid=grid, data=0, label=INFECTIOUS)
s.data[0, 0] = 0
i.data[0, 0] = 1
state = equation.get_initial_state(s, i)

storage = pde.MemoryStorage()
tracker = pde.PlotTracker(interval=10, plot_args=plot_2D_args)
solution = equation.solve(state, t_range=100, dt=1e-1, tracker=["progress", tracker, storage.tracker(interval=1)])

  0%|          | 0/100.0 [00:00<?, ?it/s]

Output()

Spent more time on handling trackers (3.84) than on the actual simulation (0.781)


In [ ]:
#save simulation as movie
pde.movie(storage=storage, filename='./movies/pde_sir_2D.mp4', progress=True, show_time=True, plot_args=plot_args)